In [1]:
import pandas as pd
import numpy as np
import os
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
import vtk
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import time
from tqdm import tqdm

In [2]:
# setup the mesh meta to handle downloads and caching
mesh_dir = 'data/cell_meshes' # or change to your desired folder
seg_source = 'graphene://https://minnie.microns-daf.com/segmentation/table/minnie65_public' # static: "precomputed://gs://iarpa_microns/minnie/minnie65/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

In [3]:
cell_id_list = [864691135841901283]

#[864691134377365225, 864691134631869459, 864691135356924298, 864691135569257151, 864691135614144971, 864691135633445828, 864691135633446340, 864691135651530692, 864691135662027204, 864691135725964092, 864691135725975612, 864691135758681856, 864691135855524524, 864691136110516855, 864691136167513207, 864691136390387071]

#[864691137019822702] #[864691135776009133]
#[864691134631869459, 864691135065127492, 864691135568924806, 864691135609473287, 864691135651530692, 864691135725964092, 864691135725975612, 864691135736897668, 864691135855524524, 864691136388245879, 864691136390387071, 864691136451061503]
#[864691136118988312]
#[864691135272287761, 864691135404198971, 864691135462779966, 864691135495288772, 864691135609956419, 864691135675330756, 864691136026290056, 864691136691582582, 864691136758228078]
#[864691135587226354]
#[864691136237449532]
#[864691135359140440]
#[864691135866387734]
#[864691135841901283] Manifest not found error
#[864691135569733766] Manifest not found error
#[864691135675330756]
#[864691135622557507]
#[864691136437464222] Manifest not found error
#[864691136781187182]
#[864691135837606792]
#[864691135695674004]
#[864691135996759498]
#[864691134964446239, 864691135012524790, 864691135256138671, 864691135269913253, 864691135337851366, 864691135341421745, 864691135348239831, 864691135386363265, 864691135497750291, 864691135657783170, 864691135809446092, 864691135837182867, 864691135880405261, 864691136023933241, 864691136194008022, 864691136330394007]
#[864691136194086870]
#[864691135081699319, 864691135628037572, 864691135724244139, 864691136134568075, 864691136577483540]
#[864691135081699319, 864691135162208557, 864691135373699401, 864691135628037572, 864691135724244139, 864691135937674805, 864691136134568075, 864691136273617421, 864691136577483540]
#[864691135269913253,864691135293126156]

In [4]:
start_time = time.time()

for i in tqdm(range(len(cell_id_list)), desc="Downloading Meshes"):
    mesh_start_time = time.time()
    
    downloadmesh = mm.mesh(seg_id=cell_id_list[i], remove_duplicate_vertices=True)
    
    mesh_elapsed_time = time.time() - mesh_start_time
    minutes, seconds = divmod(mesh_elapsed_time, 60)
    print(f"Processed mesh for cell_id {cell_id_list[i]} in {int(minutes)} minutes and {int(seconds)} seconds")

total_elapsed_time = time.time() - start_time
total_minutes, total_seconds = divmod(total_elapsed_time, 60)
print(f"Total processing time: {int(total_minutes)} minutes and {int(total_seconds)} seconds")

SSLError: None: Max retries exceeded with url: /minnie65_pcg/ws/graphene_meshes_minnie3_v1/initial/4/6298628-0.shard (Caused by None)

In [ ]:
# make a dictionary to hold the mesh file for each cell id in the list

mesh_dictionary = {}
for i in range(len(cell_id_list)):
    mesh_file = os.path.join(mesh_dir, str(cell_id_list[i]) + '_0' + '.h5')
    mesh_dictionary['cell_' + str(i)] = mm.mesh(filename = mesh_file)

In [ ]:
mesh_dictionary

In [ ]:
cell_gran = 1000
cell_opac = 1

fig, ax = plt.subplots()

for cellid in range(len(mesh_dictionary)):
    cell_key = 'cell_' + str(cellid)
    mesh = mesh_dictionary[cell_key]
    random_color = np.random.rand(3,)
    
    ax.scatter(
        mesh.vertices[::cell_gran, 0],
        mesh.vertices[::cell_gran, 1],
        s=1, 
        alpha=cell_opac, 
        color=random_color
    )

ax.invert_yaxis()  # Reverse the y-axis

plt.show()


In [ ]:
cell_gran = 1000
cell_opac = 1

fig = go.Figure()

for cellid in range(len(mesh_dictionary)):
    cell_key = 'cell_' + str(cellid)
    mesh = mesh_dictionary[cell_key]
    random_color = np.random.rand(3,)
    
    fig.add_trace(go.Scatter3d(
        x=mesh.vertices[::cell_gran, 0],
        y=mesh.vertices[::cell_gran, 1],
        z=mesh.vertices[::cell_gran, 2],
        mode='markers',
        marker=dict(size=1, opacity=cell_opac, color='rgb({}, {}, {})'.format(*random_color)),
        hoverinfo='none',
        showlegend=False
    ))

# Set the camera view
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=0.5, z=1)  # Adjust these values to match the view in your screenshot
)

fig.update_layout(
    scene=dict(
        xaxis=dict(autorange='reversed'),  # Reverse the x-axis
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.4),  # Compress the z-axis
        camera=camera
    ),
     dragmode='orbit'  # Set default interaction mode to orbital rotation
)

fig.show()


Do not run the following if there are lot of meshes or large meshes (such as astrocytes)

In [ ]:
# opacity and color settings for cell membrane 
cell_opac = 0.4
highlight_opac = 0.4
highlight_color = (0.2, 0.4, 0.7)

# cell membrane mesh

mesh_actor = {}
for i in range(len(mesh_dictionary)):
    random_color = list(np.random.random(size=3))
    mesh_actor['cell_' + str(i)] = trimesh_vtk.mesh_actor(mesh_dictionary['cell_'+str(i)], opacity=cell_opac, color=(random_color))

locals().update(mesh_actor)

# increase cell_i opacity to highlight it against other actors
cellnum = 0
mesh_actor['cell_' + str(cellnum)] = trimesh_vtk.mesh_actor(mesh_dictionary['cell_' + str(cellnum)], opacity=highlight_opac, color=(highlight_color))
locals().update(mesh_actor)

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh_dictionary['cell_'+str(cellnum)].centroid, backoff=1500)

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)

In [ ]:
# Convert each integer to a string
str_list = [str(i) for i in cell_id_list]

# Join the list into a single string with underscores
str_list_for_filename = "_".join(str_list)

# type(str_list_for_filename)

In [ ]:
# save image to disk
# uncomment to save

# camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=1500) # modify backoff accordingly

# trimesh_vtk.render_actors(mesh_actor.values(),
#                           filename='vtk_images/'+ str_list_for_filename +'.png',
#                           do_save=True,
#                           scale=6,
#                           camera=camera
#                          )